### Imports

In [1]:
import pandas as pd
import yaml
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from scenarios import ScenarioSettings
import sampling

### Read Configs

In [9]:
config_path = os.path.join(os.path.abspath(os.path.join('..', 'config.yaml')))
with open(config_path, "r") as file:
    config = yaml.safe_load(file)
SCENARIO = config['hyperparameters']['scenario']
settings = ScenarioSettings[SCENARIO].value
input_dim = settings.dim
test_function = settings.fun

### Generate Data

In [5]:
qmc_train_x = sampling.generate_sobol_sequence(
    num_samples = 8192,
    dim = input_dim
)
qmc_train_y = test_function(qmc_train_x)

mc_train_x = sampling.generate_random_sequence(
    num_samples = 8192,
    dim = input_dim
)
mc_train_y = test_function(mc_train_x)

In [6]:
qmc_train_df = pd.DataFrame(
    data=qmc_train_x,
    columns=[f"input_{i+1}" for i in range(qmc_train_x.shape[1])]
)
qmc_train_df["solution"] = qmc_train_y

mc_train_df = pd.DataFrame(
    data=mc_train_x,
    columns=[f"input_{i+1}" for i in range(mc_train_x.shape[1])]
)
mc_train_df["solution"] = mc_train_y

### Store Data

In [7]:
output_dir = os.path.join('../data', SCENARIO, 'input')
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, 'qmc_train.csv')
qmc_train_df.to_csv(output_path, index=False)

output_path = os.path.join(output_dir, 'mc_train.csv')
mc_train_df.to_csv(output_path, index=False)